In [ ]:
import pandas as pd
import json
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
# Extracts player matches from json files 
file_list = []

folder_path = "./rta_player_data"
files_in_folder = os.listdir(folder_path)

# Filter out only files (excluding directories)
files = [file for file in files_in_folder if os.path.isfile(os.path.join(folder_path, file))]

for file in files:
    file_list.append('./rta_player_data/' + file)


In [ ]:
def generate_drafts(user_data):
    battles = pd.read_json(user_data)
    battles = dict(battles.loc["battle_list"])['result_body']
    
    # holds all matches
    data_match = list()

    for x in range(len(battles)):
        
        # generic dict to store all features
        data = dict()

        # extracts hero pick order
        my_list = battles[x]['my_deck']
        enemy_list = battles[x]['enemy_deck']
        hero_deck = my_list["hero_list"]
        enemy_hero_deck = enemy_list["hero_list"]
        
        # checks for matches that do not have hero data
        # and skips that match
        team = battles[x]['teamBettleInfo']
        string_data_team = '{' + team + '}'
        data2 = json.loads(string_data_team)
        my_team_data = data2["my_team"]
        if not my_team_data:
            continue
        
        # extracts match pre bans and which side won
        my_preban = my_list['preban_list']
        enemy_preban = enemy_list['preban_list']
        is_win = battles[x]['iswin']
        
        # extract which user has the first turn
        is_first = 0
        for pick in hero_deck:
            first_pick = pick['first_pick']
            if first_pick == 1:
                is_first = 1
                break
            
        # heroes         
        for i in range(5):
            
            data['main' + str(i+1)] = hero_deck[i]['hero_code']
            data['enemy' + str(i+1)] = enemy_hero_deck[i]['hero_code']
            
        # pre-bans 
        for i in range(2):
            data['main_pre_b' + str(i+1)] = my_preban[i]
            data['enemy_pre_b' + str(i+1)] = enemy_preban[i] 
        
        # post-ban for main
        for i in range(5):
            if hero_deck[i]['ban'] == 1:
                data['main_post_b'] = str(hero_deck[i]['hero_code'])
        
        # post-ban for enemy     
        for i in range(5):
            if enemy_hero_deck[i]['ban'] == 1:
                data['enemy_post_b'] = str(enemy_hero_deck[i]['hero_code']) 
        
        # adds the user with first pick and winner to the match instance    
        data['is_first'] = is_first  
        data['is_win'] = is_win
        
        data_match.append(data)
    
    return data_match

In [ ]:
# keep track of each match
drafts_dataset = []
for i in range(len(file_list)):
    res = generate_drafts(file_list[i])
    drafts_dataset.append(res)

# turns the 2d array to 1d and converts to dataframe
collapsed_arr = [item for sublist in drafts_dataset for item in sublist]
dataset = pd.DataFrame(collapsed_arr)


In [ ]:
dataset

In [ ]:
# sampled 50 random rows (matches)
rows = dataset.sample(n=50)
dataset_sample = pd.DataFrame(rows)
dataset_sample.reset_index(drop=True, inplace=True)


In [ ]:
# checks the user whether they have the first pick and 
# keeps track of the hero picked first by that user
first_picked_col = []
for index, row in dataset_sample.iterrows():
    if row['is_first'] == 1:
        val = row['main1']
        first_picked_col.append(val)
    else:
        val = row['enemy1']
        first_picked_col.append(val)
        
dataset_sample['first_picked'] = first_picked_col

In [ ]:
# Plots most common first picks 
graphMostFirstPick = dataset_sample['first_picked'].value_counts()
graphMostFirstPick.plot(kind='bar')
plt.xlabel('Most Common First Picks')
plt.ylabel('Instance Count')
plt.title('Frequency of Matches with the Most First Picks')
plt.tight_layout()
plt.show()

In [ ]:
# Specify the file name
csv_file = 'drafts_dataset_sample.csv'

# Write the concatenated DataFrame to a CSV file
dataset_sample.to_csv(csv_file, index=False)



In [ ]:
# Specify the file name
whole_dataset_csv_file = 'drafts_dataset.csv'

# Write the concatenated DataFrame to a CSV file
dataset.to_csv(whole_dataset_csv_file, index=False)

In [ ]:
first_picked_col_all = []
for index, row in dataset.iterrows():
    if row['is_first'] == 1:
        val = row['main1']
        first_picked_col_all.append(val)
    else:
        val = row['enemy1']
        first_picked_col_all.append(val)
        
dataset['first_picked'] = first_picked_col_all

In [ ]:
graphMostFirstPick = dataset['first_picked'].value_counts()
graphMostFirstPick.plot(kind='bar')
plt.xlabel('Most Common First Picks')
plt.ylabel('Instance Count')
plt.title('Frequency of Matches with the Most First Picks')
plt.tight_layout()
plt.show()

In [162]:
recommended_fp = [] 
for i in range(10):
    recommended_fp.append(dataset['first_picked'].value_counts().index[i])

In [163]:

recommended_fp

['c1159',
 'c2089',
 'c2016',
 'c1133',
 'c2109',
 'c2046',
 'c6037',
 'c4004',
 'c2090',
 'c2008']

In [181]:
def draft_response(m1, e1, m2, e2, m3, e3,
                   m4, e4, m5, e5, mpb1, mpb2, epb1, epb2):
    
    # list to keep track of heroes to not recommend to draft
    cannot_draft = []
    response = []
    
    copy_recommended_fp = recommended_fp

    cannot_draft.append(mpb1) 
    cannot_draft.append(mpb2)
    cannot_draft.append(epb1)
    cannot_draft.append(epb2)
    
    # Main first pick
    # "first pick" placeholder for first pick 
    if m1 == "first pick":
        
        for pick in copy_recommended_fp:
            for cannotpick in cannot_draft:
                if pick == cannotpick:
                    copy_recommended_fp.remove(pick) 
                    
        # returns the next avaliable item           
        return [copy_recommended_fp[0]]
    
    # main has fp
    if m2 == _ and m3 == _:
            
        cannot_draft.append(m1)
        cannot_draft.append(e1)
        cannot_draft.append(e2)
        
        for index, row in dataset.iterrows():
            
            # enemy fp -> main 1,2 
            # enemy 1,2,3 -> main 3,4
            # enemy 1,2,3,4,5 -> main 5
            
            # m2 m3 response into e1 e2  
                
                if row['is_first'] == 0 and row['enemy1'] == e1 and row['enemy2'] == e2:
                    
                    response1 = row['main2']
                    response2 = row['main3']
                    response.append([response1, response2])
                    responses_df = pd.DataFrame(response)
                    choices = responses_df.value_counts()[:20]
                    array = [pair for pair in choices.index] 
                    
                    flattened_list = [item for sublist in array for item in sublist]
                    data_singles = list(set(flattened_list))
                    
                    for pick in data_singles:
                        for cannotpick in cannot_draft:
                            if pick == cannotpick:
                                data_singles.remove(pick)
                                
                    return data_singles[:2]
    
    if m4 == _ and m5 == _:
            
        cannot_draft.append(m1)
        cannot_draft.append(e1)
        cannot_draft.append(e2)
        cannot_draft.append(m2)
        cannot_draft.append(m3)
        cannot_draft.append(e3)
        cannot_draft.append(e4)
        
        for index, row in dataset.iterrows():
            
            # enemy fp -> main 1,2 
            # enemy 1,2,3 -> main 3,4
            # enemy 1,2,3,4,5 -> main 5
            
            # m2 m3 response into e1 e2  
                
            if row['is_first'] == 0 and row['enemy3'] == e3 and row['enemy4'] == e4:
                
                response1 = row['main4']
                response2 = row['main5']
                response.append([response1, response2])
                responses_df = pd.DataFrame(response)
                choices = responses_df.value_counts()[:20]
                array = [pair for pair in choices.index] 
                
                flattened_list = [item for sublist in array for item in sublist]
                data_singles = list(set(flattened_list))
                
                for pick in data_singles:
                    for cannotpick in cannot_draft:
                        if pick == cannotpick:
                            data_singles.remove(pick)
                            
                return data_singles[:2]
    
    
    # enemy has fp 
    
    if m1 == _ and m2 == _:
            
        cannot_draft.append(e1) 
        
        for index, row in dataset.iterrows():
        
            # input e1
            # returns m1 m2
            if row['is_first'] == 0 and row['enemy1'] == e1:
                response1 = row['main1']
                response2 = row['main2']
                response.append([response1, response2])
                responses_df = pd.DataFrame(response)
                choices = responses_df.value_counts()[:20]
                array = [pair for pair in choices.index] 
                flattened_list = [item for sublist in array for item in sublist]
                data_singles = list(set(flattened_list))
                
                for pick in data_singles:
                    for cannotpick in cannot_draft:
                        if pick == cannotpick:
                            data_singles.remove(pick)
                            
                return data_singles[:2]
        
    if m3 == _ and m4 == _:    
            
        cannot_draft.append(e1) 
        cannot_draft.append(e2) 
        cannot_draft.append(e3)
        cannot_draft.append(m1) 
        cannot_draft.append(m2)  
        
        for index, row in dataset.iterrows():
            # input e1, e2, e3
            # returns m3 m4
            if row['is_first'] == 0 and row['enemy2'] == e2 and row['enemy3'] == e3:
                response1 = row['main3']
                response2 = row['main4']
                response.append([response1, response2])
                responses_df = pd.DataFrame(response)
                choices = responses_df.value_counts()[:20]
                array = [pair for pair in choices.index] 
                flattened_list = [item for sublist in array for item in sublist]
                data_singles = list(set(flattened_list))
                
                for pick in data_singles:
                    for cannotpick in cannot_draft:
                        # if any of the picks are in the recommended, then
                        if pick == cannotpick:
                            data_singles.remove(pick)
                            
                return data_singles[:2]
            
    if m5 == _:    
        cannot_draft.append(e1) 
        cannot_draft.append(e2) 
        cannot_draft.append(e3)
        cannot_draft.append(e4)
        cannot_draft.append(e5)
        cannot_draft.append(m1) 
        cannot_draft.append(m2) 
        cannot_draft.append(m3) 
        cannot_draft.append(m4)
        for index, row in dataset.iterrows():       
            # input e1, e2, e3, e4, e5
            # returns m5
            if row['is_first'] == 0 and  row['enemy4'] == e4 and row['enemy5'] == e5:
                response1 = row['main5']
                response.append([response1])
                responses_df = pd.DataFrame(response)
                choices = responses_df.value_counts()[:20]
                array = [pair for pair in choices.index] 
                flattened_list = [item for sublist in array for item in sublist]
                data_singles = list(set(flattened_list))
                
                for pick in data_singles:
                    for cannotpick in cannot_draft:
                        if pick == cannotpick:
                            data_singles.remove(pick)
                            
                return data_singles[:1]            
                
        else:
            # only suggests for main 
            output = "Invalid Input"
            
    return output
        


In [180]:
first_pick_main = "first pick"

# idea to take in a dynamic list?
# order m1, e1, m2, e2, m3, e3, m4, e4, m5, e5,

# test case pre ban Laia, Carmin, Clilias, Candy

# input "fp"
# returns m1 
draft_response("first pick", _, _, _, _, 
               _, _, _, _, _,"c1159", "c2008", "c2089", "c2109")

['c2016']

In [177]:
# input m1, e1, e2 ()
# returns m2 m3
draft_response("c2016", "c4144", _, "c2022", _, 
               _, _, _, _, _,"c1159", "c2008", "c2089", "c2109")

['c1116']

In [72]:
# input m1,e1 e2, m2, m3, e4, e5
# returns m4, m5
# draft_response("c2016", "c4144", _, "c2022", _, 
#                _, _, _, _, _,"c1159", "c2008", "c2089", "c2109")

In [178]:
# test case pre ban Laia, Carmin, Clilias, Candy
# enemy fp ayufine
draft_response(_,"c2016", _, _, _, 
               _, _, _, _, _,"c1159", "c2008", "c2089", "c2109")

['c4144', 'c2022']

In [136]:
# test case pre ban Laia, Carmin, Clilias, Candy
# enemy fp ayufine, e2, e3
# main picks m1, m2
# returns m3, m4
draft_response("c4144","c2016", "c2022", "c1133", _, 
               "c1118", _, _, _, _,
               "c1159", "c2008", "c2089", "c2109")

['c1091', 'c1151']

In [137]:
# enemy 1-5
# main adin, destina, elena, ran
# enemy yufine, zio, nahkwol, cidd, jacko 
# returns m5
draft_response("c4144","c2016", "c2022", "c1133", "c1091", 
               "c1118", "c1151", "c1014", _, "c1130",
               "c1159", "c2008", "c2089", "c2109")

['c2020']

In [176]:
draft_response("c1159", "c2016", "c2109", "c4004", "c2008", 
               "c2022", _, "c2071", _, _,"c1151", "c2089", "c2089", "c1133")

['c5089', 'c2046']